<a href="https://colab.research.google.com/github/PedroPinheiroMachado/Portfolio/blob/main/webscrapping_buteco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sobre o Projeto

Provavelmente você já escutou falar sobre o evento "Comida di Buteco", já foi até motivo de diversas reportagens ao longo dos anos.

Este Projeto nada mais é do que webscrapping inicial dos botecos do *Rio de Janeiro* de 2025, presentes na [Página Oficial](https://comidadibuteco.com.br/butecos/rio-de-janeiro/).

Olhando um pouco a página, percebemos que temos o `nome do estabelecimento`, o `endereço`, `prato` e a `descrição do prato`.

Nosso trabalho é pegar estas informações e agregá-las em um *dataset*

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import unicodedata
import pandas as pd

##Importação dos dados

Iremos acessar a página e usando as divisões de classes do html, selecionar o que nos interessa.

Na div `caption`: temos tanto o nome quanto o endereço.
Assim, utilizaremos o separador destes de acordo com o html, (h2 e p)

Outro problema que temos é que possuímos diversas páginas para acessar, porém pode-se perceber que o domínio é estático, apenas variando no número da página, então construímos um url que muda de acordo com o número da página.

In [2]:
nome=[]
endereco=[]
for i in range(1,21):
  url= f'https://comidadibuteco.com.br/butecos/rio-de-janeiro/page/{i}/'
  page = requests.get(url)
  dadospagina = BeautifulSoup(page.text, "html.parser")
  frases = dadospagina.find_all("div", class_="caption")
  for div in frases:
    nome.append(div.find("h2").text)
    endereco.append(div.find("p").text)


In [4]:
nome[:5]

['Angurmê Culinária Afro-brasileira',
 'Antônio’s Nordestino',
 'Armazém São Thiago',
 'Art Chopp',
 'Baixo Gago']

In [5]:
endereco[:5]

['Rua Visconde de Abaeté, 123 | Vila Isabel, Rio de Janeiro - RJ',
 'Rua Ati, 365 | Tanque, Rio de Janeiro - RJ',
 'Rua Áurea, 26 | Santa Teresa, Rio de Janeiro - RJ',
 'Rua Macembu, 63 Rio de Janeiro - RJ',
 'Rua Gago Coutinho, 51 | Laranjeiras, Rio de Janeiro - RJ']

##Pegando os pratos

Agora devemos acessar cada um dos estabelecimentos, neste momento, fazemos uma análise rápida no endereço de cada boteco, por exemplo: Armazém Sçao Thiago. possui endereço como https://comidadibuteco.com.br/buteco/armazem-sao-thiago/

Veja que com o nome que já temos, conseguimos acessar a página, porém devemos tratar como pode ser visto, Os caracteres especiais, o maiúsculo, e botar os hífens.


Porém encontraremos mais um problema, alguns estabelecimentos possuem endereços diferentes dos nomes, nestes casos, iremos tratar separadamente. Aqui ocorre apenas uma vez, com o Tibar Taquara

In [ ]:
#Definir nossas listas para salvar
listafim=[]
listacheia=[]
legenda=[]

#Preparação dos nomes para botar no endereço
for i in range(0,len(nome)):
  nome1 = unicodedata.normalize('NFKD', nome[i]).encode('ASCII', 'ignore').decode('ASCII')
  nome1 = nome1.replace("ç", "c").replace("'", "").replace('"', "")
  nome1=nome1.strip().replace(" ","-")
  print(nome1)

#Tratar os "outliers"
  if nome1 == "Tibar-Taquara":
    url= "https://comidadibuteco.com.br/buteco/tibar/"
    page = requests.get(url)
    dadospagina = BeautifulSoup(page.text, "html.parser")
    frases = dadospagina.find_all("div", class_="section-text")
    for div in frases:
      listafim.append(div.find("p").find("b").text.lstrip())
      listacheia.append((div.find("p").text.lstrip()))
  else:
    url= f'https://comidadibuteco.com.br/buteco/{nome1}/'
    page = requests.get(url)
    dadospagina = BeautifulSoup(page.text, "html.parser")
    frases = dadospagina.find_all("div", class_="section-text")
    for div in frases:
      if div.find("p").find("b") == None:
        listafim.append(div.find("p").find("strong").text.lstrip())
        print("comida:")
        print(div.find("p").find("strong").text.lstrip())
        listacheia.append(div.find("p").text.lstrip())
      else:
          paragrafos = div.find("p").find("b").text
          print("comida:")
          print(div.find("p").text.lstrip())
          listacheia.append(div.find("p").text.lstrip())
          listafim.append(paragrafos)

In [8]:
listacheia[:5]


['Fofoquinha da VilaBolinho de língua desfiada e fubá mimoso empanado com flocos de milho crocante, acompanhado de vinagrete maxixe e pimenta biquinho.',
 'Harmonia do Sertão Escondidinho de aipim com carne-seca desfiada, queijo coalho, feijão-de-corda, gratinado no forno. Acompanha: manteiga de garrafa e queijo parmesão ralado.',
 'Croqueta Espetaculosa Croquetas de diferentes tipos de carne de porco: barriga de porco, pernil, blend de costelinhas e joelho de porco com geleia de bacon com cebolas e especiarias.',
 'Nosso Porquinho deu Cuscuz!Copa lombo defumado em lenha de macieira, desfiado e servido com farofa de cuscuz nordestino.Endereço: Rua Estrada Macembu, 63 | Taquara, Rio de Janeiro – RJ',
 'Flor do SolCebola empanada recheada de arroz de carne seca, coberta de queijo provolone gratinado.']

In [9]:
listafim[:5]

['Fofoquinha da Vila',
 'Harmonia do Sertão',
 'Croqueta Espetaculosa',
 'Nosso Porquinho deu Cuscuz!',
 'Flor do Sol']

##Tratar O problema do  nome do prato e legenda do prato

Como podemos ver, nossa listafim possui apenas os nomes dos pratos, porém listacheia está com o nome dos pratos e as legendas.

Infelizmente no html do site, não conseguimos acessar separadamente cada um, então devemos tentar ao máximo separar com oque temos.

In [14]:
def remove_intersecao(lista1, lista2):
    lista2_sem_interseccao = []

    for frase2 in lista2:
        for frase1 in lista1:
            # Usar regex para substituir as palavras inteiras
            frase2 = re.sub(r'\b' + re.escape(frase1) + r'\b', '', frase2)

        # Remover espaços extras e garantir que não reste nada no final
        lista2_sem_interseccao.append(frase2.strip())

    return lista2_sem_interseccao

def remove_apos_endereco(lista):
    resultado = []
    for frase in lista:
        # Usar regex para encontrar a palavra "Endereço" e tudo que vem depois
        frase_modificada = re.sub(r'Endereço.*', 'Endereço', frase)
        resultado.append(frase_modificada.strip())
    return resultado

Iremos tirar da listacheia tudo que possui o nome dos pratos, além disso, temos que no final sempre adiciona-se o endereço, assim, também iremos removê-lo.

In [15]:
legenda=remove_intersecao(listafim, listacheia)
legenda=remove_apos_endereco(legenda)

In [18]:
legenda[3:10]

['Fofoquinha da VilaBolinho de língua desfiada e fubá mimoso empanado com flocos de milho crocante, acompanhado de vinagrete maxixe e pimenta biquinho.',
 'Escondidinho de aipim com carne-seca desfiada, queijo coalho, feijão-de-corda, gratinado no forno. Acompanha: manteiga de garrafa e queijo parmesão ralado.',
 'Croquetas de diferentes tipos de carne de porco: barriga de porco, pernil, blend de costelinhas e joelho de porco com geleia de bacon com cebolas e especiarias.',
 'Copa lombo defumado em lenha de macieira, desfiado e servido com farofa de cuscuz nordestino.Endereço',
 'Flor do SolCebola empanada recheada de arroz de carne seca, coberta de queijo provolone gratinado.',
 'Bolinhos de milho com recheio de mexilhão, empanados e fritos na farinha Panko, acompanhados de um creme de queijo gorgonzola.',
 'Paçoca de carne de sol com feijão-verde debulhado e queijo coalho.',
 'Caldo de fraldinha desfiada com creme de batata e feijão-branco.',
 'Camarão que acompanha bacalhau, acorda 

In [21]:
#Verificar os tamanhos
print(len(listacheia))
print(len(legenda))
print(len(listafim))
print(len(endereco))
print(len(nome))


94
94
94
94
94


##Transformando em um dataframe

Usaremos o pandas para realizar a transformação das listas que temos em um dataframe.

In [ ]:
df = pd.DataFrame({
    'Nome do Estabelecimento': nome,
    'Endereço': endereco,
    'Prato': listafim,
    "Descrição do Prato": legenda
})

##DataFrame

Chegamos ao final, nosso *dataset* está pronto, algumas coisas que poderiamos melhorar certamente é na descrição do prato.

Este possui alguns erros como pode ser visto rapidamente, isto ocorreu pois não pude acessar melhor as divs feitas no site, assim tive que tentar separar utilizando uma própria análise textual.

Outro ponto que pode-se melhorar, é no endereço, podemos passar para lat e long, uma vez que o endereço por extenso pouco se diz.

In [ ]:
df

,Nome do Estabelecimento,Endereço,Prato,Descrição do Prato
0,Angurmê Culinária Afro-brasileira,"Rua Visconde de Abaeté, 123 | Vila Isabel, Rio...",Fofoquinha da Vila,Fofoquinha da VilaBolinho de língua desfiada e...
1,Antônio’s Nordestino,"Rua Ati, 365 | Tanque, Rio de Janeiro - RJ",Harmonia do Sertão,"Escondidinho de aipim com carne-seca desfiada,..."
2,Armazém São Thiago,"Rua Áurea, 26 | Santa Teresa, Rio de Janeiro - RJ",Croqueta Espetaculosa,Croquetas de diferentes tipos de carne de porc...
3,Art Chopp,"Rua Macembu, 63 Rio de Janeiro - RJ",Nosso Porquinho deu Cuscuz!,"Copa lombo defumado em lenha de macieira, desf..."
4,Baixo Gago,"Rua Gago Coutinho, 51 | Laranjeiras, Rio de Ja...",Flor do Sol,Flor do SolCebola empanada recheada de arroz d...
...,...,...,...,...
89,Varandão do Valqueire,"Rua Jambeiro, 433 | Rio de Janeiro - RJ",Rabada a pururuca,Rabada desfiada à pururuca acompanhada ao molh...
90,Victor Bar,"Rua Riachuelo, 32 | Centro, Rio de Janeiro - RJ",O Sabor do Sucesso,Cesta crocante recheada com lombo suíno desfia...
91,Vigal Bar,"Rua Lopo Saraiva, 179 | Tanque, Rio de Janeiro...",Porquinho na cuxxxtela da vaca,"3 bolinhos de costela cremosos, cada um rechea..."
92,Vila Rica,"Rua Cândido Mendes, 16 | Glória, Rio de Janeir...",Trio trem bão,"Três minipães de polvilho com queijo, recheado..."


In [ ]:
df.to_csv("Buteco.csv")

Agradeço a todos que estiveram até aqui.

Podem me contatar sobre quaisquer dúvidas pelo meu email: pedropm@id.uff.br.